<a href="https://colab.research.google.com/github/YShu7/FYP/blob/master/exp/LSTM_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import numpy as np
import sqlite3
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
 
torch.manual_seed(1)
torch.set_deterministic(True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
 
# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 32
HIDDEN_DIM = 128
INPUT_DIM = 10
TRAIN_DB = 'database_100+_train_2.db'
TEST_DB = 'database_100-_test_2.db'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/torch/__init__.py:422: UserWarning: torch.set_deterministic is deprecated and will be removed in a future release. Please use torch.use_deterministic_algorithms instead
  "torch.set_deterministic is deprecated and will be removed in a future "


In [ ]:
device

device(type='cuda')

# Dataset

In [ ]:
class PathDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, inputs, targets, output_size, transform=None):
        self.inputs = inputs
        self.targets = targets
        self.output_size = output_size
        self.transform = transform

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        inputs = torch.tensor(self.inputs[idx], dtype=torch.float).to(device)
        targets = torch.tensor(self.targets[idx], dtype=torch.float).to(device)
        sample = (inputs, targets)
        if self.transform:
            sample = self.transform(sample)

        return sample

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def prepare_data(df_contacts, to_ix):
  real_ids = set(df_contacts['real_id'].tolist())
  input, target = [], []
  # For each words-list (sentence) and tags-list in each tuple of training_data

  for real_id in real_ids:
    walk = df_contacts.loc[df_contacts['real_id'] == real_id].sort_values('time')
    path, times = [], []

    for i, row in walk.iterrows():
      times.append(int(row['time']))
      path.append(to_ix[row['agent_id']])

    start_idx = 0
    while True:
      try:
        # first_time = times[start_idx]
        curr_input = path[start_idx:start_idx+10]
        curr_target = path[start_idx+10]
        start_idx += INPUT_DIM

        input.append(curr_input)
        target.append(curr_target)
      except:
        break

  print(len(input), len(target))

  return input, target

In [ ]:
N, M = 30, 91
cnx = sqlite3.connect(f'/content/drive/MyDrive/FYP/{TRAIN_DB}')

df_walks = pd.read_sql_query("SELECT * FROM walks JOIN walkers ON walkers.id = walks.walker_id", cnx)
df_walkers = pd.read_sql_query("SELECT * FROM walkers", cnx)
df_agents = pd.read_sql_query("SELECT * FROM agents", cnx)
df_contacts_tmp = pd.read_sql_query("SELECT * FROM contacts JOIN walkers ON walkers.id = contacts.walker_id", cnx)
idx = df_contacts_tmp.groupby(['walker_id', 'time'])['distance'].transform(min) == df_contacts_tmp['distance']
df_contacts = df_contacts_tmp[idx]

agent_to_ix = {}
for _, row in df_agents.iterrows():
  x, y = row['id'].split('EVEN')
  agent_to_ix[row['id']] = [int(x)/N, (int(y)%M)/M]

transform = None
print(agent_to_ix)

input_train, target_train = prepare_data(df_contacts, agent_to_ix)
train_loader = torch.utils.data.DataLoader(PathDataset(input_train, target_train, len(agent_to_ix), transform),
                                           batch_size=64, shuffle =True)

{'0EVEN0': [0.0, 0.0], '0EVEN1': [0.0, 0.01098901098901099], '0EVEN2': [0.0, 0.02197802197802198], '0EVEN3': [0.0, 0.03296703296703297], '0EVEN4': [0.0, 0.04395604395604396], '0EVEN5': [0.0, 0.054945054945054944], '0EVEN6': [0.0, 0.06593406593406594], '0EVEN7': [0.0, 0.07692307692307693], '0EVEN9': [0.0, 0.0989010989010989], '0EVEN11': [0.0, 0.12087912087912088], '0EVEN13': [0.0, 0.14285714285714285], '0EVEN14': [0.0, 0.15384615384615385], '0EVEN15': [0.0, 0.16483516483516483], '0EVEN16': [0.0, 0.17582417582417584], '0EVEN17': [0.0, 0.18681318681318682], '0EVEN18': [0.0, 0.1978021978021978], '0EVEN19': [0.0, 0.2087912087912088], '0EVEN20': [0.0, 0.21978021978021978], '0EVEN21': [0.0, 0.23076923076923078], '0EVEN22': [0.0, 0.24175824175824176], '0EVEN23': [0.0, 0.25274725274725274], '0EVEN24': [0.0, 0.26373626373626374], '0EVEN25': [0.0, 0.27472527472527475], '0EVEN26': [0.0, 0.2857142857142857], '0EVEN27': [0.0, 0.2967032967032967], '0EVEN28': [0.0, 0.3076923076923077], '0EVEN29': [0.0

In [ ]:
cnx = sqlite3.connect(f'/content/drive/MyDrive/FYP/{TEST_DB}')

df_walks = pd.read_sql_query("SELECT * FROM walks JOIN walkers ON walkers.id = walks.walker_id", cnx)
df_walkers = pd.read_sql_query("SELECT * FROM walkers", cnx)
df_agents = pd.read_sql_query("SELECT * FROM agents", cnx)
df_contacts_tmp = pd.read_sql_query("SELECT * FROM contacts JOIN walkers ON walkers.id = contacts.walker_id", cnx)
idx = df_contacts_tmp.groupby(['walker_id', 'time'])['distance'].transform(min) == df_contacts_tmp['distance']
df_contacts = df_contacts_tmp[idx]

input_val, target_val = prepare_data(df_contacts[:2000], agent_to_ix)
val_loader = torch.utils.data.DataLoader(PathDataset(input_val, target_val, len(agent_to_ix), transform),
                                         batch_size=1, shuffle=False)

input_test, target_test = prepare_data(df_contacts[2000:], agent_to_ix)
test_loader = torch.utils.data.DataLoader(PathDataset(input_test, target_test, len(agent_to_ix), transform), 
                                          batch_size=1, shuffle =False)

193 193
1653 1653


# Model

In [ ]:
DEBUG = False
def evaluate(model, loader, DEBUG=False):
  model.train(False)

  running_loss = 0
  running_corrects = 0

  for data in loader:

    inputs, targets = data
    outputs = model(inputs.type(torch.FloatTensor).to(device)).to(device) #forward pass
    
    loss = criterion(outputs, targets.type(torch.FloatTensor).to(device))

    running_loss += loss.item()

    for pred, target in zip(outputs, targets):
      x_pred, x_target = round(float(pred[0])*N), round(float(target[0])*N)
      y_pred = round(float(pred[1])*M) + x_pred*M
      y_target = round(float(target[1]*M)) + x_target*M
      if x_pred == x_target and \
        y_pred == y_target:
        running_corrects += 1
  
  epoch_loss = running_loss / len(loader)
  epochs_acc = running_corrects / len(loader) * 100

  # print(x_pred, y_pred, x_target, y_target)

  return epoch_loss, epochs_acc

#### Single

In [ ]:
DEBUG = False
class LSTMTagger(nn.Module):

  def __init__(self, input_dim, embedding_dim, hidden_dim, num_classes, num_layers=1, seq_len=10):
    super(LSTMTagger, self).__init__()
    self.hidden_dim = hidden_dim
    self.num_layers = num_layers
    self.embedding_dim = embedding_dim


    # LSTM
    self.lstm = nn.LSTM(2, hidden_dim, num_layers=num_layers, batch_first=True) #lstm
    self.lstm_1 = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
    self.fc_1 = nn.Linear(hidden_dim, hidden_dim)
    self.fc = nn.Linear(hidden_dim, 2)
    self.relu = nn.ReLU() 

  def forward(self, input):
    # INPUT: (batch, seq_len)
    if DEBUG: print('input: ', input.shape)

    # LSTM
    # LSTM: (batch, seq, feature)
    lstm, (hn, cn) = self.lstm(input)
    lstm = nn.Dropout(0.2)(lstm)
    if DEBUG: print('lstm:', lstm.shape)

    lstm_1, (hn, cn) = self.lstm_1(lstm)
    hn = nn.Dropout(0.2)(hn)
    if DEBUG: print('hn:', hn.shape)
    # hn = torch.cat((hn[-2,:,:], hn[-1,:,:]), dim = 1)
    hn = hn.view(-1, self.hidden_dim)
    if DEBUG: print('hn:', hn.shape)

    out = self.fc_1(hn).to(device) #first Dense
    if DEBUG: print('fc_1:', out.shape)
    out = self.relu(out).to(device) #relu

    out = self.fc(out).to(device)

    return out


#### Bidirectional

In [ ]:
# DEBUG = False
# class LSTMTagger(nn.Module):

#   def __init__(self, input_dim, embedding_dim, hidden_dim, num_classes, num_layers=1, seq_len=10):
#     super(LSTMTagger, self).__init__()
#     self.hidden_dim = hidden_dim
#     self.num_layers = num_layers
#     self.embedding_dim = embedding_dim


#     # LSTM
#     self.lstm = nn.LSTM(2, hidden_dim, num_layers=num_layers, bidirectional=True, batch_first=True) #lstm
#     self.lstm_1 = nn.LSTM(hidden_dim*2, hidden_dim, bidirectional=True, batch_first=True)
#     self.fc_1 = nn.Linear(hidden_dim*2, hidden_dim)
#     self.fc = nn.Linear(hidden_dim, 2)
#     self.relu = nn.ReLU() 

#   def forward(self, input):
#     # INPUT: (batch, seq_len)
#     if DEBUG: print('input: ', input.shape)

#     # LSTM
#     # LSTM: (batch, seq, feature)
#     lstm, (hn, cn) = self.lstm(input)
#     lstm = nn.Dropout(0.2)(lstm)
#     if DEBUG: print('lstm:', lstm.shape)

#     lstm_1, (hn, cn) = self.lstm_1(lstm)
#     hn = nn.Dropout(0.2)(hn)
#     if DEBUG: print('hn:', hn.shape)
#     hn = torch.cat((hn[-2,:,:], hn[-1,:,:]), dim = 1)
#     # hn = hn.view(-1, self.hidden_dim)
#     if DEBUG: print('hn:', hn.shape)

#     out = self.fc_1(hn).to(device) #first Dense
#     if DEBUG: print('fc_1:', out.shape)
#     out = self.relu(out).to(device) #relu

#     out = self.fc(out).to(device)

#     return out


In [ ]:
train_loader

# Train

In [ ]:
model = LSTMTagger(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, len(agent_to_ix)).to(device)
print(model)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1000):  # again, normally you would NOT do 300 epochs, it is toy data
  model.train(True)
  for data in train_loader:
    model.zero_grad()

    inputs, targets = data
    outputs = model(inputs.type(torch.FloatTensor).to(device)).to(device) #forward pass
    loss = criterion(outputs, targets.type(torch.FloatTensor).to(device)) #obtain the loss function
    loss.backward() #calculates the loss of the loss function
    optimizer.step() #improve from loss, i.e backprop
    
  if (epoch + 1) % 50 == 0:
    epoch_loss, epochs_acc = evaluate(model, val_loader)
    print("Epoch: %d, loss: %1.5f, acc: %1.5f" % (epoch+1, epoch_loss, epochs_acc))
    
    torch.save(model.state_dict(), f'e{epoch+1}.pkl')


LSTMTagger(
  (lstm): LSTM(2, 128, batch_first=True)
  (lstm_1): LSTM(128, 128, batch_first=True)
  (fc_1): Linear(in_features=128, out_features=128, bias=True)
  (fc): Linear(in_features=128, out_features=2, bias=True)
  (relu): ReLU()
)
Epoch: 50, loss: 0.00095, acc: 43.00518
Epoch: 100, loss: 0.00073, acc: 46.63212
Epoch: 150, loss: 0.00068, acc: 9.32642
Epoch: 200, loss: 0.00059, acc: 49.74093
Epoch: 250, loss: 0.00050, acc: 55.44041
Epoch: 300, loss: 0.00035, acc: 47.15026
Epoch: 350, loss: 0.00045, acc: 39.37824
Epoch: 400, loss: 0.00034, acc: 57.51295
Epoch: 450, loss: 0.00030, acc: 52.84974
Epoch: 500, loss: 0.00027, acc: 45.07772
Epoch: 550, loss: 0.00054, acc: 47.66839
Epoch: 600, loss: 0.00027, acc: 58.54922
Epoch: 650, loss: 0.00065, acc: 57.51295
Epoch: 700, loss: 0.00074, acc: 58.54922
Epoch: 750, loss: 0.00071, acc: 51.29534
Epoch: 800, loss: 0.00071, acc: 53.36788
Epoch: 850, loss: 0.00052, acc: 55.95855
Epoch: 900, loss: 0.00072, acc: 54.92228
Epoch: 950, loss: 0.00068

In [ ]:
model = LSTMTagger(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, len(agent_to_ix)).to(device)
model.load_state_dict(torch.load('e950.pkl'))
evaluate(model, test_loader, True)
torch.save(model.state_dict(), 'e950_regre_2.pkl',_use_new_zipfile_serialization=False)

In [ ]:
INTERVAL = 60
TIME_PERIOD = 1
SCALE_METERS = 15000
scaleMeters = SCALE_METERS * 0.2
SIGMA_V = scaleMeters / 100
IMPOSSIBLE = 1e-10
MINUTE = 10
UPDATE_TIME = MINUTE * INTERVAL

In [ ]:
walker_ids = set(df_contacts['walker_id'].tolist())
link_list = {}
for walker_id in walker_ids:
    last_time = df_contacts.loc[df_contacts['walker_id'] == walker_id].sort_values('time').iloc[-1]['time']
    candidates = df_contacts.loc[(df_contacts['walker_id'] != walker_id) &
                                 (pd.to_numeric(df_contacts["time"]) == int(last_time) + INTERVAL)]
    if candidates.shape[0] == 1:
        link_list[walker_id] = candidates.iloc[-1]['walker_id']
print(len(walker_ids))

2076


In [ ]:
# """
# Connect each linked path into a single path
# """
import networkx as nx

graph = nx.Graph([(i, j) for i, j in link_list.items() if i and j])
connected_components = sorted(nx.connected_components(graph), key=len, reverse=True)
print(len(connected_components))

tried = 0
for i, component in enumerate(connected_components):
    component = list(component)
    # print(component)
    new_id = component[0]
    time_set = set()
    for c in component[1:]:
        rows = df_contacts.loc[df_contacts['walker_id'] == c]
            
        tried += df_contacts.loc[df_contacts['walker_id'] == c].shape[0]
        df_contacts = df_contacts.replace({'walker_id': {c: new_id}})
        

139


In [ ]:
def prepare_data_test(walk, to_ix):
    path = []

    for i, row in walk.iterrows():
        path.append(to_ix[row['agent_id']])

    return torch.Tensor([path]).type(torch.FloatTensor)

In [ ]:
import math
walker_ids = set(df_contacts['walker_id'])
print(len(walker_ids))
 
def compute_proba(walk, agent_to_ix, candidates):
    candidate_agent_probs = {}
    
    output = model(prepare_data_test(walk, agent_to_ix))[0] #forward pass
    output = [round(float(output[0])*30), round(float(output[1])*30) + round(float(output[0])*30)*30]
 
    for _, candidate in candidates.iterrows():
        x, y = candidate['agent_id'].split('EVEN')
        prob = 0.5 - 0.05 * (math.sqrt((int(x) - output[0])**2 + (int(y) - output[1])**2) - 1)
        # if candidate['agent_id'] == f'{output[0]}EVEN{output[1]}':
        prob = max(prob, 1e-10)
        candidate_agent_probs[candidate['walker_id']] = math.log(prob)
            
    return candidate_agent_probs

1565


In [ ]:
from scipy.optimize import linear_sum_assignment
import math
 
DEBUG = False
"""
Map agent id to its most possible next agent.
"""
model = LSTMTagger(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, len(agent_to_ix)).to(device)
model.load_state_dict(torch.load('e300_10.pkl'))
 
visited_walker_ids = set()
for i, walker_id1 in enumerate(walker_ids):
    if walker_id1 in visited_walker_ids:
        continue
    if i % 499 == 0:
        print(f'{i+1}...')
    walk = df_contacts.loc[df_contacts['walker_id'] == walker_id1].sort_values('time').iloc[-10:]
 
    time = int(walk.iloc[-1]['time'])
    # get candidate points by time
    candidates = df_contacts.loc[(pd.to_numeric(df_contacts["time"]) == time + INTERVAL)
                                 & (df_contacts['walker_id'] != walker_id1)]
 
    if candidates.shape[0] == 0:
        print('no candidates')
        continue
 
    if candidates.shape[0] == 1:
        print('1 candidates')
        link_list[walker_id] = candidates.iloc[-1]['walker_id']
        visited_walker_ids.add(walker_id)
        continue
    
 
    if walk.shape[0] != 10:
        continue
        
    raw_batch_walker_ids = set(df_contacts.loc[df_contacts['time'] == str(time)]['walker_id'])
    batch_walker_ids_prob = {}
    for walker_id2 in raw_batch_walker_ids:
        walk2 = df_contacts.loc[df_contacts['walker_id'] == walker_id2].sort_values('time').iloc[-10:]
        if int(walk2.iloc[-1]['time']) == time:
            batch_walker_ids_prob[walker_id2] = compute_proba(walk, agent_to_ix, candidates)
 
    # print(batch_walker_ids_prob)
    walker_id_to_index, walker_index_to_id = {}, []
    candidate_id_to_index, candidate_index_to_id = {}, []
 
    for k, values in batch_walker_ids_prob.items():
        walker_id_to_index[k] = len(walker_id_to_index)
        walker_index_to_id.append(k)
        for c, v in values.items():
            if c not in candidate_id_to_index:
                candidate_id_to_index[c] = len(candidate_id_to_index)
                candidate_index_to_id.append(c)
    matrix = [[math.log(1e-10)] * len(candidate_id_to_index) for _ in range(len(walker_id_to_index))]
    
    for k, values in batch_walker_ids_prob.items():
        for c, v in values.items():
            matrix[walker_id_to_index[k]][candidate_id_to_index[c]] = v
                
    # print(matrix)
    row_ind, col_ind = linear_sum_assignment(matrix)
    
    for row_idx, col_idx in zip(row_ind, col_ind):
        link_list[walker_index_to_id[row_idx]] = candidate_index_to_id[col_idx]
        visited_walker_ids.add(walker_index_to_id[row_idx])

FileNotFoundError: ignored

In [ ]:
"""
Connect each linked path into a single path
"""
graph = nx.Graph([(i, j) for i, j in link_list.items() if i and j])
connected_components = sorted(nx.connected_components(graph), key=len, reverse=True)
print(len(connected_components))
print(connected_components)
 
tried = 0
df_walks_connected = pd.DataFrame(df_walks)
for i, component in enumerate(connected_components):
    new_id = i
    time_set = set()
    for c in component:
        rows = df_walks_connected.loc[df_walks_connected['walker_id'] == c]
            
        tried += df_walks_connected.loc[df_walks_connected['walker_id'] == c].shape[0]
        df_walks_connected = df_walks_connected.replace({'walker_id': {c: str(new_id) + 'res'}})

In [ ]:
"""
How much percentage of the grouped ids are really from one group
"""
import collections
DEBUG = True
walker_ids = set(df_walks_connected['walker_id'].tolist())
correct, total = 0, 0
if DEBUG: print(len(walker_ids))
    
for walker_id in walker_ids:
    id_list = df_walks_connected.loc[df_walks_connected['walker_id'] == walker_id].sort_values('walk_time')['real_id'].tolist()
    dic = collections.Counter(id_list)
    
    l = max(dic.values())
    key = [k for k,v in dic.items() if v == l]
    
    correct += l
    total += sum(dic.values())
 
# print(f'correct/tried: {correct}/{tried} {correct/tried*100}%') 
print(f'correct/total: {correct}/{total} {correct/total*100}%')

In [ ]:
"""
How much percentage of one group is correctly grouped together (only consider the largest sub-group of grouped ids)
"""
 
DEBUG = False
real_ids = set(df_walks_connected['real_id'].tolist())
correct, total = 0, 0
for real_id in real_ids:
    id_list = df_walks_connected.loc[df_walks_connected['real_id'] == real_id].sort_values('walk_time')['walker_id'].tolist()
    dic = collections.Counter(id_list)
    if DEBUG: print(id_list)
    
    l = max(dic.values())
    key = [k for k,v in dic.items() if v == l]
    
    correct += l
    total += sum([v for v in dic.values()])
#     df_walks_connected = df_walks_connected.replace({'walker_id': {key[0]: real_id}})
# print(f'base: {total - (150 / 10 - 1) * 10}/{total} {(total - (150 / 10 - 1) * 10)/total*100}')
# print(f'correct/tried: {correct}/{tried} {correct/tried*100}%') 
print(f'correct/total: {correct}/{total} {correct/total*100}%')